## BUILDING RNN FROM SCRATCH :

In [2]:
import numpy as np

In [8]:


def softmax(x):
    e_x = np.exp(x - np.max(x))  
    return e_x / np.sum(e_x)

# Forward Propagation

In [ ]:
def single_step_rnn(xt, a_prev,Wax,Waa,Wya,ba,by):
    a_next= np.tah(np.dot(Wax,xt)+np.dot(Waa,a_prev)+ba)
    yt=softmax(np.dot(Wya,a_next)+by)
   
    cache = (a_next, a_prev, xt, Wax,Waa,Wya,ba,by)

    return a_next,yt,cache


In [11]:
def rnn_forward(X,a0,Wax,Waa,Wya,ba,by):
    n_x,m,T=X.shape
    n_y, n_a = Wya.shape
    a=np.zeros((n_a,m,T))
    y_pred=np.zeros((n_y,m,T))

    a_next=a0
    caches=[]
    for t in range(T):
        xt=X[:,:,t]
        a_next,yt,cache=single_step_rnn(xt,a_next,Wax,Waa,Wya,ba,by)
        a[:,:,t]=a_next
        y_pred[:,:,t]=yt
        caches.append(cache)
        

    caches = (caches, X)
    
    return a, y_pred, caches

    


## Backpropagation through time

In [ ]:
def single_step_backward(da_next,cache):
    a_next,a_prev,xt,Wax,Waa,Wya,ba,by=cache

    dtanh= da_next*(1-np.tanh(a_next)**2)
    dWax = np.dot(dtanh, xt.T)        
    dWaa = np.dot(dtanh, a_prev.T)
    dba  = np.sum(dtanh, axis=1, keepdims=True) 
    dxt  = np.dot(Wax.T, dtanh)    
    da_prev = np.dot(Waa.T, dtanh) 

    gradients = {
        "dxt": dxt,
        "da_prev": da_prev,
        "dWax": dWax,
        "dWaa": dWaa,
        "dba": dba
    }
    
    return gradients


In [ ]:
def rnn_backward(da, caches):
    
    
    (list_caches, x) = caches  
    (a1, a0, x1,Wax,Waa,Wya,ba,by) = list_caches[0]  

    
    n_a, m, T = da.shape       
    n_x, _ = x1.shape             
    
   
    dx    = np.zeros((n_x, m, T))
    dWax  = np.zeros((n_a, n_x))
    dWaa  = np.zeros((n_a, n_a))
    dba   = np.zeros((n_a, 1))
    da0   = np.zeros((n_a, m))
    

    da_prevt = np.zeros((n_a, m))
    
    
    for t in reversed(range(T)):
        
    
        da_current = da[:, :, t] + da_prevt
        
        
        gradients = single_step_backward(da_current, list_caches[t])
        
    
        dxt     = gradients["dxt"]
        da_prevt = gradients["da_prev"]
        dWaxt   = gradients["dWax"]
        dWaat   = gradients["dWaa"]
        dbat    = gradients["dba"]
        
    
        dx[:, :, t] = dxt
        
        
        dWax += dWaxt
        dWaa += dWaat
        dba  += dbat
    
    
    da0 = da_prevt
    
   
    gradients = {
        "dx": dx,
        "da0": da0,
        "dWax": dWax,
        "dWaa": dWaa,
        "dba": dba
    }
    
    return gradients
